In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
%matplotlib inline

import seaborn as sns
from datetime import datetime, timedelta

In [4]:
#Lister les data presentent dans le dossier
#Lecture des données 

path_data = '../../data/'

print(os.listdir(path_data))

path_ = path_data + 'glucose_12-10-2019_export.csv'

df_full = pd.read_csv(path_, sep=';')

df_full.head(5)

['.DS_Store', 'glucose_12-10-2019_export.csv', '.ipynb_checkpoints']


,Horodatage de l'appareil,Type d'enregistrement,Historique de la glycémie mg/dL,Numérisation de la glycémie mg/dL,Insuline à action rapide sans valeur numérique,Insuline à action rapide (unités),Alimentation sans valeur numérique,Glucides (grammes),Glucides (portions),Insuline à action longue sans valeur numérique,Insuline à action longue (unités),Remarques,Glycémie par bandelette mg/dL,Cétone mmol/L,Insuline repas (unités),Correction insuline (unités),Insuline modifiée par l'utilisateur (unités)
0,12-04-2019 02:26 PM,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12-04-2019 02:27 PM,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12-04-2019 03:16 PM,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12-04-2019 03:17 PM,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12-04-2019 05:15 PM,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_full['Date'] = pd.to_datetime(df_full['Horodatage de l\'appareil'])
df_full['DateCp'] = df_full['Date']
df_full = df_full.sort_values(by='Date',  ascending=False)

### Récupération des données suivantes :

- Heure du repas
- Type de repas (dej, lunch, dinner)
- Unité Lente (Insuline)
- Unité Rapide (Insuline)
- Taux de glycémie (T0)
- Taux de glycémie (T-1H)
- Taux de glycémie (T-2H)
- Taux de glycémie (T-3H)
- Taux de glycémie (T-4H)

In [6]:
#Extraction des données de glucose, heure de repas, et taux d'insuline en fonction des données freestyle
df_glucose = df_full[df_full['Type d\'enregistrement'] == 0]
df_repas = df_full[df_full['Type d\'enregistrement'] == 5]
df_insuline = df_full[df_full['Type d\'enregistrement'] == 4]

df_insuline = df_insuline[['Date', 'DateCp', 'Insuline à action longue (unités)', 'Insuline à action rapide (unités)']]
df_repas = df_repas[['Type d\'enregistrement', 'Horodatage de l\'appareil' , 'Date', 'DateCp']]

df_glucose.set_index('Date', inplace=True)
df_repas.set_index('Date', inplace=True)
df_insuline.set_index('Date', inplace=True)


df_glucose['Day'] = df_glucose['DateCp'].apply(lambda x : x.date())
df_repas['Day'] = df_repas['DateCp'].apply(lambda x : x.date())
df_insuline['Day'] = df_insuline['DateCp'].apply(lambda x : x.date())

df_glucose['Time'] = df_glucose['DateCp'].apply(lambda x : x.time())
df_repas['Time'] = df_repas['DateCp'].apply(lambda x : x.time())
df_insuline['Time'] = df_insuline['DateCp'].apply(lambda x : x.time())

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [16]:
#Pour les repas du matin et du soir on regroupe l'insuline rapide et lente
df_insuline = df_insuline.groupby('Date').first()

In [17]:
df_insuline.head()

,DateCp,Insuline à action longue (unités),Insuline à action rapide (unités),Day,Time
Date,,,,,
2017-03-20 12:32:00,2017-03-20 12:32:00,NaN,9.0,2017-03-20,12:32:00
2017-03-20 15:56:00,2017-03-20 15:56:00,NaN,2.0,2017-03-20,15:56:00
2017-03-20 20:38:00,2017-03-20 20:38:00,NaN,12.0,2017-03-20,20:38:00
2017-03-21 07:34:00,2017-03-21 07:34:00,NaN,14.0,2017-03-21,07:34:00
2017-03-21 12:33:00,2017-03-21 12:33:00,NaN,12.0,2017-03-21,12:33:00


In [20]:
df_repas_insuline = pd.merge(df_insuline,df_repas, how='inner', on='DateCp')

df_repas_insuline.drop(['Day_x', 'Time_x', 'Type d\'enregistrement', 'Horodatage de l\'appareil'], axis=1, inplace=True)

In [38]:
df_repas_insuline['Date0H'] = df_repas_insuline['DateCp']
df_repas_insuline['Date-1H'] = df_repas_insuline['DateCp'].apply(lambda x : (x - timedelta(hours=1)))
df_repas_insuline['Date-2H'] = df_repas_insuline['DateCp'].apply(lambda x : (x - timedelta(hours=2)))
df_repas_insuline['Date-3H'] = df_repas_insuline['DateCp'].apply(lambda x : (x - timedelta(hours=3)))
df_repas_insuline['Date-4H'] = df_repas_insuline['DateCp'].apply(lambda x : (x - timedelta(hours=4)))

In [39]:
df_repas_insuline.head()

,DateCp,Insuline à action longue (unités),Insuline à action rapide (unités),Day_y,Time_y,Date0H,Date-1H,Date-2H,Date-3H,Date-4H
0,2019-06-14 09:03:00,16.0,16.0,2019-06-14,09:03:00,2019-06-14 09:03:00,2019-06-14 08:03:00,2019-06-14 07:03:00,2019-06-14 06:03:00,2019-06-14 05:03:00
1,2019-06-14 13:21:00,NaN,6.0,2019-06-14,13:21:00,2019-06-14 13:21:00,2019-06-14 12:21:00,2019-06-14 11:21:00,2019-06-14 10:21:00,2019-06-14 09:21:00
2,2019-06-15 08:52:00,14.0,13.0,2019-06-15,08:52:00,2019-06-15 08:52:00,2019-06-15 07:52:00,2019-06-15 06:52:00,2019-06-15 05:52:00,2019-06-15 04:52:00
3,2019-06-15 12:49:00,NaN,6.0,2019-06-15,12:49:00,2019-06-15 12:49:00,2019-06-15 11:49:00,2019-06-15 10:49:00,2019-06-15 09:49:00,2019-06-15 08:49:00
4,2019-06-16 08:51:00,14.0,13.0,2019-06-16,08:51:00,2019-06-16 08:51:00,2019-06-16 07:51:00,2019-06-16 06:51:00,2019-06-16 05:51:00,2019-06-16 04:51:00


In [40]:
#Glucose mg/dL 
#Recuperation du taux de glucose au moment du repas
df_repas_insuline['Glucose0H'] = df_repas_insuline['Date0H'].apply(lambda x: df_glucose.iloc[df_glucose.index.get_loc(x,method='nearest')]['Historique de la glycémie mg/dL'])
df_repas_insuline['Glucose-1H'] = df_repas_insuline['Date-1H'].apply(lambda x: df_glucose.iloc[df_glucose.index.get_loc(x,method='nearest')]['Historique de la glycémie mg/dL'])
df_repas_insuline['Glucose-2H'] = df_repas_insuline['Date-2H'].apply(lambda x: df_glucose.iloc[df_glucose.index.get_loc(x,method='nearest')]['Historique de la glycémie mg/dL'])
df_repas_insuline['Glucose-3H'] = df_repas_insuline['Date-3H'].apply(lambda x: df_glucose.iloc[df_glucose.index.get_loc(x,method='nearest')]['Historique de la glycémie mg/dL'])
df_repas_insuline['Glucose-4H'] = df_repas_insuline['Date-4H'].apply(lambda x: df_glucose.iloc[df_glucose.index.get_loc(x,method='nearest')]['Historique de la glycémie mg/dL'])

In [46]:
df_repas_insuline.drop(['Date0H', 'Date-1H', 'Date-2H', 'Date-3H', 'Date-4H'], axis=1, inplace=True)
df_repas_insuline.rename(columns={"Day_y": "Day", "Time_y": "Time","Insuline à action longue (unités)": "Lente", "Insuline à action rapide (unités)": "Rapide" }, inplace=True)
df_repas_insuline.head()

,DateCp,Lente,Rapide,Day,Time,Glucose0H,Glucose-1H,Glucose-2H,Glucose-3H,Glucose-4H
0,2019-06-14 09:03:00,16.0,16.0,2019-06-14,09:03:00,108.0,101.0,100.0,105.0,110.0
1,2019-06-14 13:21:00,NaN,6.0,2019-06-14,13:21:00,124.0,144.0,194.0,163.0,104.0
2,2019-06-15 08:52:00,14.0,13.0,2019-06-15,08:52:00,67.0,73.0,74.0,83.0,88.0
3,2019-06-15 12:49:00,NaN,6.0,2019-06-15,12:49:00,46.0,58.0,101.0,86.0,67.0
4,2019-06-16 08:51:00,14.0,13.0,2019-06-16,08:51:00,109.0,92.0,86.0,78.0,77.0


In [47]:
def getRepas(timestamp_):
    
    current_time = timestamp_.time()
        
    hour_max_breakfast = datetime.strptime('11::00::00', '%H::%M::%S').time()
    hour_max_lunch = datetime.strptime('16::00::00', '%H::%M::%S').time()
    hour_max_dinner = datetime.strptime('21::00::00', '%H::%M::%S').time()
    
    if current_time < hour_max_breakfast:
        return 0
    elif current_time > hour_max_breakfast and current_time < hour_max_lunch:
        return 1
    elif current_time > hour_max_lunch and current_time < hour_max_dinner:
        return 2
    else:
        return 3

#determine le type de repas (dej, lunch, dinner)
df_repas_insuline['RepasType'] = df_repas_insuline['DateCp'].apply(lambda x: getRepas(x))

In [48]:
df_repas_insuline.head()

,DateCp,Lente,Rapide,Day,Time,Glucose0H,Glucose-1H,Glucose-2H,Glucose-3H,Glucose-4H,RepasType
0,2019-06-14 09:03:00,16.0,16.0,2019-06-14,09:03:00,108.0,101.0,100.0,105.0,110.0,0
1,2019-06-14 13:21:00,NaN,6.0,2019-06-14,13:21:00,124.0,144.0,194.0,163.0,104.0,1
2,2019-06-15 08:52:00,14.0,13.0,2019-06-15,08:52:00,67.0,73.0,74.0,83.0,88.0,0
3,2019-06-15 12:49:00,NaN,6.0,2019-06-15,12:49:00,46.0,58.0,101.0,86.0,67.0,1
4,2019-06-16 08:51:00,14.0,13.0,2019-06-16,08:51:00,109.0,92.0,86.0,78.0,77.0,0


In [50]:
# Y = Glucose 2h après le repas

df_repas_insuline['Glucose+2H'] = df_repas_insuline['DateCp'] \
                                .apply(lambda x: df_glucose.iloc[df_glucose.index.get_loc(x + timedelta(hours=2),method='nearest')]
                                       ['Historique de la glycémie mg/dL'])

In [52]:
df_repas_insuline.drop(['DateCp'], axis=1, inplace=True)

df_repas_insuline.head()

,Lente,Rapide,Day,Time,Glucose0H,Glucose-1H,Glucose-2H,Glucose-3H,Glucose-4H,RepasType,Glucose+2H
0,16.0,16.0,2019-06-14,09:03:00,108.0,101.0,100.0,105.0,110.0,0,197.0
1,NaN,6.0,2019-06-14,13:21:00,124.0,144.0,194.0,163.0,104.0,1,145.0
2,14.0,13.0,2019-06-15,08:52:00,67.0,73.0,74.0,83.0,88.0,0,101.0
3,NaN,6.0,2019-06-15,12:49:00,46.0,58.0,101.0,86.0,67.0,1,88.0
4,14.0,13.0,2019-06-16,08:51:00,109.0,92.0,86.0,78.0,77.0,0,156.0
